In [1]:
import numpy as np
np.set_printoptions(linewidth=150, suppress=True, precision=3)

In [2]:
def compute_v(V_in):
    V = V_in.copy()
    norm = np.linalg.norm(V)
    x0 = V[0].copy()
    y = - np.sign(x0) * norm
    tau = (y - x0) / y
    V[0] = y
    V[1:] /= (x0 - y)
    
    return tau, V

In [12]:
matA = np.zeros((6, 6), dtype=np.complex)
matA[0,0] = complex(-0.679585,0)
matA[1,0] = complex(0.753421,-0.529475)
matA[0,1] = complex(0.753421,0.529475)
matA[1,1] = complex(-0.0791742,0)
matA[2,0] = complex(0.582536,0.227978)
matA[3,0] = complex(0.296187,0.55139)
matA[2,1] = complex(0.478532,0.365537)
matA[3,1] = complex(-0.213113,-0.36713)
matA[4,0] = complex(0.0198112,0.168763)
matA[5,0] = complex(0.14048,-0.61663)
matA[4,1] = complex(-0.191346,0.847631)
matA[5,1] = complex(-0.726808,0.0910724)
matA[0,2] = complex(0.582536,-0.227978)
matA[1,2] = complex(0.478532,-0.365537)
matA[0,3] = complex(0.296187,-0.55139)
matA[1,3] = complex(-0.213113,0.36713)
matA[2,2] = complex(-0.305542,0)
matA[3,2] = complex(0.895346,0.19707)
matA[2,3] = complex(0.895346,-0.19707)
matA[3,3] = complex(-0.00935277,0)
matA[4,2] = complex(0.275293,-0.0185519)
matA[5,2] = complex(-0.700053,-0.425955)
matA[4,3] = complex(0.293019,0.0454628)
matA[5,3] = complex(0.0373272,0.239061)
matA[0,4] = complex(0.0198112,-0.168763)
matA[1,4] = complex(-0.191346,-0.847631)
matA[0,5] = complex(0.14048,0.61663)
matA[1,5] = complex(-0.726808,-0.0910724)
matA[2,4] = complex(0.275293,0.0185519)
matA[3,4] = complex(0.293019,-0.0454628)
matA[2,5] = complex(-0.700053,0.425955)
matA[3,5] = complex(0.0373272,-0.239061)
matA[4,4] = complex(0.175663,0)
matA[5,4] = complex(0.355474,0.0861055)
matA[4,5] = complex(0.355474,-0.0861055)
matA[5,5] = complex(-0.147128,0)

Z = np.zeros((6, 6), dtype=np.complex)
Z[0,0] = complex(-0.679585,0)
Z[1,0] = complex(0.753421,-0.529475)
Z[0,1] = complex(0.753421,0.529475)
Z[1,1] = complex(-0.0791742,0)
Z[2,0] = complex(-1.10087,-0)
Z[3,0] = complex(0.216335,0.298246)
Z[2,1] = complex(-0.0704359,0.248602)
Z[3,1] = complex(1.32954,0)
Z[4,0] = complex(0.0248886,0.0968801)
Z[5,0] = complex(0.0332341,-0.370799)
Z[4,1] = complex(-0.0423654,-0.472152)
Z[5,1] = complex(0.379258,-0.308427)
Z[0,2] = complex(0.582536,-0.227978)
Z[1,2] = complex(0.478532,-0.365537)
Z[0,3] = complex(0.296187,-0.55139)
Z[1,3] = complex(-0.213113,0.36713)
Z[2,2] = complex(0.605108,0)
Z[3,2] = complex(-0.388362,-0.109381)
Z[2,3] = complex(0.895346,-0.19707)
Z[3,3] = complex(-0.27069,0)
Z[4,2] = complex(0.817691,0)
Z[5,2] = complex(-0.552992,0.169521)
Z[4,3] = complex(0.277551,-0.446662)
Z[5,3] = complex(-0.520172,0.215131)
Z[0,4] = complex(0.0198112,-0.168763)
Z[1,4] = complex(-0.191346,-0.847631)
Z[0,5] = complex(0.14048,0.61663)
Z[1,5] = complex(-0.726808,-0.0910724)
Z[2,4] = complex(0.275293,0.0185519)
Z[3,4] = complex(0.293019,-0.0454628)
Z[2,5] = complex(-0.700053,0.425955)
Z[3,5] = complex(0.0373272,-0.239061)
Z[4,4] = complex(-0.754793,0)
Z[5,4] = complex(0.0424268,0.0343959)
Z[4,5] = complex(0.355474,-0.0861055)
Z[5,5] = complex(0.134015,0)

REF = matA.copy()

nb = 2

In [13]:
matA = REF.copy()

Vs = []
Ts = []

for j_panel in range(nb, matA.shape[0], nb):
    Ai = matA[j_panel:, j_panel-nb:j_panel]
    At = matA[j_panel:, j_panel:]
    
    display(matA)
    display('Ai', Ai)
    display('At', At)
    
    # REFLECTOR PANEL
    T = np.zeros((nb, nb), dtype=matA.dtype)
    V = np.zeros((Ai.shape[0], nb), dtype=matA.dtype)
    V[:nb,:nb] = np.eye(nb)
    
    for j in range(nb):
        Vj = Ai[j:,j]
        
        if len(Vj) == 1:
            break

        tau, Vj = compute_v(Vj)
        
        # Store tau in the T-factor
        T[j, j] = tau
        
        # Update "temp" band matrix
        Ai[j,j] = Vj[0]
        Ai[j+1:,j] = 0
        
        # Setup H matrix with householder reflectors
        V[j, j] = 1
        V[j + 1:, j] = Vj[1:].copy()
   
        # copy the current reflector
        Vj = V[j:,j].copy().reshape(-1, 1)
        
        # COMPUTE TRAILING PANEL (IF THERE IS ONE)
        if nb - (j + 1) > 0:
            Pt = Ai[j:, j + 1:]
            
            W = Pt.conj().T @ Vj
            Pt = Pt - np.conj(tau) * Vj @ W.conj().T # TODO tau conj
            
            Ai[j:, j + 1:] = Pt
        
    # Update band matrix
    matA[j_panel:, j_panel-nb:j_panel] = Ai
        
    # COMPUTE T-FACTOR
    for j in range(1, nb):
        T[0:j, j] = - T[j, j] * T[0:j, 0:j] @ V[j:, 0:j].conj().T @ V[j:, j]

    Vs.append(V)
    Ts.append(T)
        
    display('V', V)
    display('T', T)
    
    # UPDATE TRAILING MATRIX
    W = V @ T
    display('W', W, 'At', At)
    X = At @ W
    display('X\'', X)
    W2 = W.conj().T @ X
    display('W2', W2)
    X = X - .5 * V @ W2
    display('X', X)

    At = At - X @ V.conj().T - V @ X.conj().T
    matA[j_panel:, j_panel:] = At
    
    display('At', At)

display(matA)

array([[-0.68 +0.j   ,  0.753+0.529j,  0.583-0.228j,  0.296-0.551j,  0.02 -0.169j,  0.14 +0.617j],
       [ 0.753-0.529j, -0.079+0.j   ,  0.479-0.366j, -0.213+0.367j, -0.191-0.848j, -0.727-0.091j],
       [ 0.583+0.228j,  0.479+0.366j, -0.306+0.j   ,  0.895-0.197j,  0.275+0.019j, -0.7  +0.426j],
       [ 0.296+0.551j, -0.213-0.367j,  0.895+0.197j, -0.009+0.j   ,  0.293-0.045j,  0.037-0.239j],
       [ 0.02 +0.169j, -0.191+0.848j,  0.275-0.019j,  0.293+0.045j,  0.176+0.j   ,  0.355-0.086j],
       [ 0.14 -0.617j, -0.727+0.091j, -0.7  -0.426j,  0.037+0.239j,  0.355+0.086j, -0.147+0.j   ]])

'Ai'

array([[ 0.583+0.228j,  0.479+0.366j],
       [ 0.296+0.551j, -0.213-0.367j],
       [ 0.02 +0.169j, -0.191+0.848j],
       [ 0.14 -0.617j, -0.727+0.091j]])

'At'

array([[-0.306+0.j   ,  0.895-0.197j,  0.275+0.019j, -0.7  +0.426j],
       [ 0.895+0.197j, -0.009+0.j   ,  0.293-0.045j,  0.037-0.239j],
       [ 0.275-0.019j,  0.293+0.045j,  0.176+0.j   ,  0.355-0.086j],
       [-0.7  -0.426j,  0.037+0.239j,  0.355+0.086j, -0.147+0.j   ]])

...

'X'

array([[0.322+0.07j , 0.   +0.j   ],
       [0.514-0.221j, 0.   +0.j   ]])

'At'

array([[-0.755+0.j   ,  0.042-0.034j],
       [ 0.042+0.034j,  0.134+0.j   ]])

array([[-0.68 +0.j   ,  0.753+0.529j,  0.583-0.228j,  0.296-0.551j,  0.02 -0.169j,  0.14 +0.617j],
       [ 0.753-0.529j, -0.079+0.j   ,  0.479-0.366j, -0.213+0.367j, -0.191-0.848j, -0.727-0.091j],
       [-1.101-0.j   , -0.07 +0.249j,  0.605+0.j   , -0.388+0.109j, -0.397-0.115j,  0.652+0.27j ],
       [ 0.   +0.j   ,  1.33 +0.j   , -0.388-0.109j, -0.271+0.j   , -0.549-0.347j, -0.198+0.364j],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.818+0.j   ,  0.278-0.447j, -0.755+0.j   ,  0.042-0.034j],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   , -0.52 +0.215j,  0.042+0.034j,  0.134+0.j   ]])

In [14]:
display('Z', Z)
display('matA', matA)
display('diff', np.tril(matA - Z))

'Z'

array([[-0.68 +0.j   ,  0.753+0.529j,  0.583-0.228j,  0.296-0.551j,  0.02 -0.169j,  0.14 +0.617j],
       [ 0.753-0.529j, -0.079+0.j   ,  0.479-0.366j, -0.213+0.367j, -0.191-0.848j, -0.727-0.091j],
       [-1.101+0.j   , -0.07 +0.249j,  0.605+0.j   ,  0.895-0.197j,  0.275+0.019j, -0.7  +0.426j],
       [ 0.216+0.298j,  1.33 +0.j   , -0.388-0.109j, -0.271+0.j   ,  0.293-0.045j,  0.037-0.239j],
       [ 0.025+0.097j, -0.042-0.472j,  0.818+0.j   ,  0.278-0.447j, -0.755+0.j   ,  0.355-0.086j],
       [ 0.033-0.371j,  0.379-0.308j, -0.553+0.17j , -0.52 +0.215j,  0.042+0.034j,  0.134+0.j   ]])

'matA'

array([[-0.68 +0.j   ,  0.753+0.529j,  0.583-0.228j,  0.296-0.551j,  0.02 -0.169j,  0.14 +0.617j],
       [ 0.753-0.529j, -0.079+0.j   ,  0.479-0.366j, -0.213+0.367j, -0.191-0.848j, -0.727-0.091j],
       [-1.101-0.j   , -0.07 +0.249j,  0.605+0.j   , -0.388+0.109j, -0.397-0.115j,  0.652+0.27j ],
       [ 0.   +0.j   ,  1.33 +0.j   , -0.388-0.109j, -0.271+0.j   , -0.549-0.347j, -0.198+0.364j],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.818+0.j   ,  0.278-0.447j, -0.755+0.j   ,  0.042-0.034j],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   , -0.52 +0.215j,  0.042+0.034j,  0.134+0.j   ]])

'diff'

array([[ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [-0.   -0.j   , -0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [-0.216-0.298j,  0.   +0.j   ,  0.   +0.j   , -0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [-0.025-0.097j,  0.042+0.472j, -0.   +0.j   ,  0.   +0.j   , -0.   +0.j   ,  0.   +0.j   ],
       [-0.033+0.371j, -0.379+0.308j,  0.553-0.17j ,  0.   +0.j   ,  0.   +0.j   , -0.   +0.j   ]])

In [15]:
for j in range(matA.shape[1]):
    for i in range(j+1, matA.shape[0]):
        matA[j, i] = np.conj(matA[i, j])
display(matA)

array([[-0.68 +0.j   ,  0.753+0.529j, -1.101+0.j   ,  0.   -0.j   ,  0.   -0.j   ,  0.   -0.j   ],
       [ 0.753-0.529j, -0.079+0.j   , -0.07 -0.249j,  1.33 -0.j   ,  0.   -0.j   ,  0.   -0.j   ],
       [-1.101-0.j   , -0.07 +0.249j,  0.605+0.j   , -0.388+0.109j,  0.818-0.j   ,  0.   -0.j   ],
       [ 0.   +0.j   ,  1.33 +0.j   , -0.388-0.109j, -0.271+0.j   ,  0.278+0.447j, -0.52 -0.215j],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.818+0.j   ,  0.278-0.447j, -0.755+0.j   ,  0.042-0.034j],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   , -0.52 +0.215j,  0.042+0.034j,  0.134+0.j   ]])

In [16]:
V = np.zeros((Vs[0].shape[0], len(Vs) * nb), dtype=Vs[0].dtype)
for i, Hi in enumerate(Vs):
    V[V.shape[0] - Hi.shape[0]:, i * nb: (i + 1) * nb] = Hi
display(V)

array([[ 1.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.216+0.298j,  1.   +0.j   ,  0.   +0.j   ,  0.   +0.j   ],
       [ 0.025+0.097j, -0.042-0.472j,  1.   +0.j   ,  0.   +0.j   ],
       [ 0.033-0.371j,  0.379-0.308j, -0.553+0.17j ,  1.   +0.j   ]])

In [17]:
taus = [tau for T in Ts for tau in T.diagonal()]
display(taus)

[(1.5291584213123797+0.20708845217111685j),
 (1.2233844121532873+0.418305390227304j),
 (1.48526614298368-0.14097409510101125j),
 0j]

In [18]:
B = matA.copy()

for j in range(V.shape[1]-1, -1, -1):
    vj = V[j:, j]
    n = vj.shape[0]
    
    Qj = np.eye(n) - taus[j] * np.outer(vj, vj.conj().T)
    
    H = np.eye(B.shape[0], dtype=B.dtype)
    H[-n:,-n:] = Qj
    
    B = H @ B @ H.conj().T

display('A = Q B Q*', B)

'A = Q B Q*'

array([[-0.68 +0.j   ,  0.753+0.529j,  0.583-0.228j,  0.296-0.551j,  0.02 -0.169j,  0.14 +0.617j],
       [ 0.753-0.529j, -0.079+0.j   ,  0.479-0.366j, -0.213+0.367j, -0.191-0.848j, -0.727-0.091j],
       [ 0.583+0.228j,  0.479+0.366j, -0.306+0.j   ,  0.895-0.197j,  0.275+0.019j, -0.7  +0.426j],
       [ 0.296+0.551j, -0.213-0.367j,  0.895+0.197j, -0.009+0.j   ,  0.293-0.045j,  0.037-0.239j],
       [ 0.02 +0.169j, -0.191+0.848j,  0.275-0.019j,  0.293+0.045j,  0.176-0.j   ,  0.355-0.086j],
       [ 0.14 -0.617j, -0.727+0.091j, -0.7  -0.426j,  0.037+0.239j,  0.355+0.086j, -0.147-0.j   ]])

In [19]:
display('REF', REF)
display('A = Q B Q*', B)
display('diff', REF - B)

'REF'

array([[-0.68 +0.j   ,  0.753+0.529j,  0.583-0.228j,  0.296-0.551j,  0.02 -0.169j,  0.14 +0.617j],
       [ 0.753-0.529j, -0.079+0.j   ,  0.479-0.366j, -0.213+0.367j, -0.191-0.848j, -0.727-0.091j],
       [ 0.583+0.228j,  0.479+0.366j, -0.306+0.j   ,  0.895-0.197j,  0.275+0.019j, -0.7  +0.426j],
       [ 0.296+0.551j, -0.213-0.367j,  0.895+0.197j, -0.009+0.j   ,  0.293-0.045j,  0.037-0.239j],
       [ 0.02 +0.169j, -0.191+0.848j,  0.275-0.019j,  0.293+0.045j,  0.176+0.j   ,  0.355-0.086j],
       [ 0.14 -0.617j, -0.727+0.091j, -0.7  -0.426j,  0.037+0.239j,  0.355+0.086j, -0.147+0.j   ]])

'A = Q B Q*'

array([[-0.68 +0.j   ,  0.753+0.529j,  0.583-0.228j,  0.296-0.551j,  0.02 -0.169j,  0.14 +0.617j],
       [ 0.753-0.529j, -0.079+0.j   ,  0.479-0.366j, -0.213+0.367j, -0.191-0.848j, -0.727-0.091j],
       [ 0.583+0.228j,  0.479+0.366j, -0.306+0.j   ,  0.895-0.197j,  0.275+0.019j, -0.7  +0.426j],
       [ 0.296+0.551j, -0.213-0.367j,  0.895+0.197j, -0.009+0.j   ,  0.293-0.045j,  0.037-0.239j],
       [ 0.02 +0.169j, -0.191+0.848j,  0.275-0.019j,  0.293+0.045j,  0.176-0.j   ,  0.355-0.086j],
       [ 0.14 -0.617j, -0.727+0.091j, -0.7  -0.426j,  0.037+0.239j,  0.355+0.086j, -0.147-0.j   ]])

'diff'

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.-0.j,  0.-0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -0.-0.j,  0.+0.j,  0.-0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-0.j,  0.-0.j, -0.+0.j],
       [ 0.+0.j, -0.+0.j,  0.+0.j,  0.-0.j,  0.+0.j, -0.-0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.-0.j, -0.+0.j,  0.+0.j],
       [ 0.-0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.-0.j, -0.+0.j]])